In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


df_train = pd.read_csv( "train.csv" )
df_test = pd.read_csv( "test.csv" )


In [2]:
df_all = pd.concat( [ df_train.drop( ['fraud_ind' ], axis = 1 ),
                      df_test ],
                      axis = 0, ignore_index = False )
df_all.shape

(1943452, 22)

In [3]:
# df_all.stscd = df_all.stscd.astype( str )
# df_all.iterm = df_all.iterm.astype( str )
# df_all.hcefg = df_all.hcefg.astype( str )
# df_all.etymd = df_all.etymd.astype( str )
# df_all.csmcu = df_all.csmcu.astype( str )
# df_all.contp = df_all.contp.astype( str )

In [4]:
cols1 = ["flbmk" , "flg_3dsmk"]
for col in cols1:
    df_all[col].fillna("N", inplace=True)

In [5]:
df_all = df_all.drop(['cano'], axis = 1)
# print( df_all.shape)

In [6]:
# df_all = df_all.drop(['iterm'], axis = 1)
# print( df_all.shape)

In [7]:
# df_all = df_all.drop(['bacno'], axis = 1)
# print( df_all.shape)

In [8]:
df_all = df_all.drop(['txkey'], axis = 1)
print( df_all.shape)

(1943452, 20)


In [9]:
# df_all = df_all.drop(['bacno'], axis = 1)
# print( df_all.shape)

In [10]:
# df_all = df_all.drop(['acqic'], axis = 1)
# print( df_all.shape)

In [11]:
df_all.loctm=df_all.loctm.astype(str)
df_all['loctm'] = df_all.loctm.str.extract('([0-9]+)\.', expand = False)
df_all['loctm']=df_all.loctm.str.rjust(6,fillchar='0')
df_all['loctm']=df_all.loctm.str.slice(0,2)
df_all.loctm=df_all.loctm.astype(int)

In [12]:
df_all_dum = pd.get_dummies(df_all); df_all_dum.head()

,acqic,bacno,conam,contp,csmcu,etymd,hcefg,iterm,locdt,loctm,...,ecfg_N,ecfg_Y,flbmk_N,flbmk_Y,flg_3dsmk_N,flg_3dsmk_Y,insfg_N,insfg_Y,ovrlt_N,ovrlt_Y
0,6881,113261,513.80,5,0,0,5,0,33,17,...,1,0,1,0,1,0,1,0,1,0
1,0,134508,465.62,5,0,2,0,0,9,10,...,1,0,1,0,1,0,1,0,1,0
2,6881,15408,513.80,5,0,0,5,0,6,15,...,1,0,1,0,1,0,1,0,1,0
3,6716,157159,1016.11,5,62,5,5,0,5,17,...,1,0,1,0,1,0,1,0,1,0
4,5975,105985,713.66,5,62,4,5,0,6,18,...,1,0,1,0,1,0,1,0,1,0


In [13]:
# normalization helps a bit
from sklearn import preprocessing
df_all_dum[ 'conam' ] = preprocessing.scale( df_all_dum[ 'conam' ] )
df_all_dum[ 'conam' ].head()

0   -0.341794
1   -0.461282
2   -0.341794
3    0.903959
4    0.153869
Name: conam, dtype: float64

## Training Start

In [14]:
# inputs
from sklearn.model_selection import train_test_split
X = df_all_dum[:df_train.shape[0]]
y = df_train.fraud_ind

y.shape

(1521787,)

In [15]:
X, X_val, y, y_val = train_test_split(X, y, test_size = 0.3, random_state = 40)

X_test = df_all_dum[ df_train.shape[0]: ]

In [16]:
# from xgboost import XGBClassifier
# from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
# from sklearn.metrics import accuracy_score
# warnings.filterwarnings('ignore')

# parameters= {'n_estimators':[100,200,500], 'learning_rate':[0.1,0.05,0.01,0.001], 'max_depth':[8,10,12]}
# xgb_model = XGBClassifier()
# xgb_model.fit(X, y)
# xgb_best_model = RandomizedSearchCV(xgb_model, parameters, cv=5, scoring='accuracy', return_train_score='cv_results_')
# xgb_best_model.fit(X, y)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier

In [18]:
warnings.filterwarnings('ignore')
# parameters= { 'max_depth': [5, 10, 15, 20, 25],
#               'learning_rate': [0.1, 0.3,0.5],
# #               'n_estimators': [500, 1000, 2000, 3000, 5000],
#               'min_child_weight': [0, 2, 5, 10, 20],
#               'max_delta_step': [0, 0.2, 0.6, 1, 2],
#               'subsample': [0.6, 0.7, 0.8, 0.85, 0.95],
#               'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9],
#               'reg_alpha': [0, 0.25, 0.5, 0.75, 1],
#               'reg_lambda': [0.2, 0.4, 0.6, 0.8, 1],
#               'scale_pos_weight': [0.2, 0.4, 0.6, 0.8, 1,73]}
parameters={
#             'n_estimators':range(80,200,4),
            'max_depth':[24,48],
            'learning_rate':[0.1,0.3,0.5],
            'subsample':[0.5,1],
            'colsample_bytree':[0.5,1],
            'min_child_weight':[1,5,10]}


xgb_model = XGBClassifier(tree_method='gpu_hist', objective='binary:logistic'
                          ,n_estimators=50
                          ,eval_metric="error", early_stopping_rounds=10)
# xgb_model.fit(X, y)
xgb_best_model =GridSearchCV(xgb_model, parameters, cv=3, scoring='f1',verbose=2, n_jobs=3)
xgb_best_model.fit(X, y)

print("Best score: %0.3f" % xgb_best_model.best_score_)
print("Best parameters set:")
best_parameters = xgb_best_model.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))


Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  7.0min
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed: 29.5min
[Parallel(n_jobs=3)]: Done 216 out of 216 | elapsed: 38.8min finished


Best score: 0.714
Best parameters set:
	colsample_bytree: 1
	learning_rate: 0.3
	max_depth: 48
	min_child_weight: 1
	subsample: 1


In [19]:
vclf = VotingClassifier(estimators=[('xgb', xgb_best_model)])

In [20]:
# from sklearn.metrics import accuracy_score
# xgb_best_model.score(X_test, y_test)

In [22]:
vclf.fit(X, y)

TypeError: fit() got an unexpected keyword argument 'n_jobs'

In [ ]:
from sklearn.metrics import f1_score
val_pre=vclf.predict(X_val)
print(f1_score(val_pre, y_val))

In [ ]:
prediction=vclf.predict(X_test)
# mapping={0:1,1:2,2:3,3:4,4:5,5:6,6:7}
# prediction=[mapping[y] for y in prediction]

data_submission=pd.DataFrame()
data_submission["txkey"]=df_test['txkey']
data_submission["fraud_ind"]=prediction
data_submission.to_csv("xgb_submission.csv",index=False)